In [223]:
## Importing modules

In [224]:
import pandas as pd
import os

In [225]:
## Setting path

In [226]:
pathdata = '/home/danish/Documents/movielens/data'
os.getcwd()
os.chdir(pathdata)

In [227]:
## Creating the movie dataframe and cleaning it

In [228]:
headername = ("movie id", "movie title", "release date", "video release date", "IMDb URL", "unknown", "Action",
              "Adventure", "Animation",
              "Children's", "Comedy","Crime", "Documentary", "Drama","Fantasy",
              "Film-Noir" ,"Horror", "Musical","Mystery","Romance", "Sci-Fi",
              "Thriller", "War", "Western")

In [229]:
moviedata = pd.read_csv("u.item", header=None, engine="python", names = headername, sep="|")

In [230]:
moviedata = moviedata.drop(["video release date" , "IMDb URL"], axis=1)
moviedata = pd.melt(moviedata, id_vars=["movie id", "movie title", "release date"], var_name = "Genre")
moviedata = moviedata[moviedata.value != 0]
moviedata = moviedata.drop(columns="value")
moviedata = moviedata.sort_values(by="movie id")
moviedata['release date'] =  pd.to_datetime(moviedata['release date'], format= '%d-%b-%Y')
## This dataframe below is joining the genres
moviedata2 = moviedata.groupby(['movie id','movie title','release date'])['Genre'].apply(', '.join).reset_index()

In [231]:
## Creating the Ratings Dataframe and cleaning it

In [232]:
heads = ("user id","movie id","rating","timestamp")
rating1 = pd.read_csv("u.data", sep="\t" , header=None, engine="python", names = heads)
rating = rating1.drop(columns="timestamp")
rating = rating.drop(columns="user id")
rating["avg rating"]= rating.groupby("movie id")["rating"].transform("mean")
rating = rating.drop(columns="rating")
rating["total number of rating"] = rating.groupby("movie id")["movie id"].transform("count")
rating = rating.drop_duplicates()
rating = rating.sort_values(by="movie id", inplace=False)
rating = rating.reset_index(drop=True)
rating["avg rating"] = rating['avg rating'].round()

In [233]:
## Merging the two dataframe

In [234]:
data = pd.merge(left=moviedata2, right=rating, on="movie id")

In [235]:
 ## Question 1 Print a list of the 10 movies that received the most number of ratings. 

In [236]:
df1 = data.sort_values(by="total number of rating", inplace=False, ascending=False)
print(df1["movie title"].head(10))

49                  Star Wars (1977)
257                   Contact (1997)
99                      Fargo (1996)
180        Return of the Jedi (1983)
292                 Liar Liar (1997)
284      English Patient, The (1996)
286                    Scream (1996)
0                   Toy Story (1995)
298             Air Force One (1997)
120    Independence Day (ID4) (1996)
Name: movie title, dtype: object


In [237]:
## Question 2 
## Print a list of the 10 movies that received the most number of ratings, sorted by the number of ratings. 

In [238]:
print(df1[["movie title", "total number of rating"]].head(10))

                       movie title  total number of rating
49                Star Wars (1977)                     583
257                 Contact (1997)                     509
99                    Fargo (1996)                     508
180      Return of the Jedi (1983)                     507
292               Liar Liar (1997)                     485
284    English Patient, The (1996)                     481
286                  Scream (1996)                     478
0                 Toy Story (1995)                     452
298           Air Force One (1997)                     431
120  Independence Day (ID4) (1996)                     429


In [239]:
## Question 3
## Print a list of the number of ratings received by each genre. 

In [240]:
data2 = pd.merge(left=moviedata, right=rating, on="movie id")
data3 = data2[["Genre", "total number of rating"]]
data3.groupby("Genre").sum()

,total number of rating
Genre,
Action,25589
Adventure,13753
Animation,3605
Children's,7182
Comedy,29832
Crime,8055
Documentary,758
Drama,39895
Fantasy,1352


In [241]:
## Question 4
## Print the oldest movie with a “5” rating. 

In [242]:
old = pd.merge(left=moviedata2, right=rating1, on="movie id")
old = old[old["rating"] == 5]
old = old.sort_values(by="release date", ascending=True)
print("the oldest movie with a 5 rating is: " , old["movie title"][0:1].to_string(index=False))

the oldest movie with a 5 rating is:  Nosferatu (Nosferatu, eine Symphonie des Graue...


In [243]:
## Question 5
## Print a list of the genre of the top 10 most rated movies. 

In [244]:
top10 = data2.sort_values(by=("total number of rating"), inplace=False, ascending=False)
top10 = top10.head(10)
top10genre = top10["Genre"].tolist()
top10genre
#for line in top10genre:

['Adventure',
 'Action',
 'Sci-Fi',
 'War',
 'Romance',
 'Sci-Fi',
 'Drama',
 'Crime',
 'Thriller',
 'Drama']

In [245]:
## importing user data and cleaning it for next question

In [246]:
usertitles = ("user id","age","gender","occupation","zip code")
users = pd.read_csv("u.user", sep="|" , header=None, engine="python", names = usertitles)   

In [247]:
fulluserinfo = pd.merge(left=users, right=rating1, on="user id")
studentsonly = fulluserinfo[fulluserinfo["occupation"] == "student"]
studentsonly = studentsonly.reset_index(drop = True)
students = studentsonly[["movie id", "rating"]].copy()
students["avg rating"]= students.groupby("movie id")["rating"].transform("mean")
students = students.drop(columns="rating")
students["total number of rating"] = students.groupby("movie id")["movie id"].transform("count")
students = students.drop_duplicates()
students = students.sort_values(by="movie id", inplace=False)
students = students.reset_index(drop=True)
students["avg rating"] = students['avg rating'].round()

In [248]:
## Question 6 Print the title of the movie that was rated the most by students 

In [249]:
studentdata = pd.merge(left=moviedata2, right=students, on="movie id")
studentdata = studentdata.sort_values(by="total number of rating", ascending=False)
studentdata.head(1)

,movie id,movie title,release date,Genre,avg rating,total number of rating
282,288,Scream (1996),1996-12-20,"Thriller, Horror",4.0,137


In [250]:
## Question 7 
## Print the list of movies that received the highest number of “5” rating 

In [251]:
all = pd.merge(left=moviedata2, right=rating1, on="movie id")
all = all[all["rating"] == 5]
all.tail()
all["total number of rating"] = all.groupby("movie id")["movie id"].transform("count")
all = all.drop(columns = ["user id", "timestamp"])
all = all.drop_duplicates()
all = all.sort_values(by="total number of rating", ascending=False)
all.head(10)

,movie id,movie title,release date,Genre,rating,total number of rating
6402,50,Star Wars (1977),1977-01-01,"Romance, Adventure, Action, Sci-Fi, War",5,325
14385,100,Fargo (1996),1997-02-14,"Crime, Thriller, Drama",5,227
17883,127,"Godfather, The (1972)",1972-01-01,"Drama, Action, Crime",5,214
24548,174,Raiders of the Lost Ark (1981),1981-01-01,"Adventure, Action",5,202
7539,56,Pulp Fiction (1994),1994-01-01,"Crime, Drama",5,188
49322,318,Schindler's List (1993),1993-01-01,"Drama, War",5,186
13823,98,"Silence of the Lambs, The (1991)",1991-01-01,"Drama, Thriller",5,181
48593,313,Titanic (1997),1997-01-01,"Romance, Drama, Action",5,179
23857,172,"Empire Strikes Back, The (1980)",1980-01-01,"War, Romance, Action, Adventure, Sci-Fi, Drama",5,172
26164,181,Return of the Jedi (1983),1997-03-14,"Sci-Fi, War, Romance, Action, Adventure",5,171


In [252]:
## Question 8
## Print the list of zip codes corresponding to the highest number of users that rated movies. 

In [253]:
zip = fulluserinfo['zip code'].value_counts()
zip.head(10)

55414    1103
20009     878
10019     850
22902     832
61820     817
48103     746
10003     736
60657     685
80525     678
83702     639
Name: zip code, dtype: int64

In [254]:
## Preparing data for next question

In [255]:
youngpeople = fulluserinfo[fulluserinfo["age"].between(20,25 ,inclusive=True)]
youngpeople = youngpeople[["movie id", "rating"]].copy()
youngpeople["avg rating"]= youngpeople.groupby("movie id")["rating"].transform("mean")
youngpeople = youngpeople.drop(columns="rating")
youngpeople["total number of rating"] = youngpeople.groupby("movie id")["movie id"].transform("count")
youngpeople = youngpeople.drop_duplicates()
youngpeople = youngpeople.sort_values(by="movie id", inplace=False)
youngpeople = youngpeople.reset_index(drop=True)
youngpeople["avg rating"] = youngpeople['avg rating'].round()

In [256]:
## Question 9
## Find the most rated movie by users in the age group 20 to 25. 

In [262]:
youngdata = pd.merge(left=moviedata2, right=youngpeople, on="movie id")
youngdata = youngdata.sort_values(by="total number of rating", ascending=False)
youngdata.head(1)

,movie id,movie title,release date,Genre,avg rating,total number of rating
286,288,Scream (1996),1996-12-20,"Thriller, Horror",4.0,130


In [258]:
## Question 10
## Print the list of movies that were rate after year 1960. 

In [259]:
data[(data["release date"] > "1960-01-01")]

,movie id,movie title,release date,Genre,avg rating,total number of rating
0,1,Toy Story (1995),1995-01-01,"Children's, Animation, Comedy",4.0,452
1,2,GoldenEye (1995),1995-01-01,"Action, Adventure, Thriller",3.0,131
2,3,Four Rooms (1995),1995-01-01,Thriller,3.0,90
3,4,Get Shorty (1995),1995-01-01,"Comedy, Drama, Action",4.0,209
4,5,Copycat (1995),1995-01-01,"Crime, Drama, Thriller",3.0,86
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,1995-01-01,Drama,4.0,26
6,7,Twelve Monkeys (1995),1995-01-01,"Drama, Sci-Fi",4.0,392
7,8,Babe (1995),1995-01-01,"Comedy, Drama, Children's",4.0,219
8,9,Dead Man Walking (1995),1995-01-01,Drama,4.0,299
9,10,Richard III (1995),1996-01-22,"War, Drama",4.0,89
